# Build a Retrieval Augmented Generation (RAG) App
Este archivo es una prueba siguiendo el tutorial descripto en la página: https://python.langchain.com/docs/tutorials/rag/

# Installation

```console
py venv -m venv
.\\venv\scripts\activate
pip install --quiet --upgrade langchain langchain-community langchain-chroma
pip install -qU langchain-openai
pip install bs4
pip install lxml
pip install pypdf
```

## LangSmith
Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.

In [1]:
import os
import utils

os.environ["LANGCHAIN_TRACING_V2"] = utils.config["LANG"]["LANGCHAIN_TRACING_V2"]
os.environ["LANGCHAIN_API_KEY"] = utils.config["LANG"]["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_PROJECT"] = "RAG-PMX"


## 1. Indexing: Load
We need to first load the blog post contents. We can use DocumentLoaders for this, which are objects that load in data from a source and return a list of Documents. A Document is an object with some page_content (str) and metadata (dict).

In this case we’ll use the WebBaseLoader, which uses urllib to load HTML from web URLs and BeautifulSoup to parse it to text. We can customize the HTML -> text parsing by passing in parameters into the BeautifulSoup parser via bs_kwargs (see BeautifulSoup docs). In this case only HTML tags with class “post-content”, “post-title”, or “post-header” are relevant, so we’ll remove all others.

In [2]:
from glob import glob
from langchain.document_loaders import BSHTMLLoader, PyPDFLoader

# Ruta con un patrón de búsqueda para archivos HTML
html_files = glob("C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es/*.html")
pdf_files = glob("C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/*.pdf")

# Cargar y procesar cada archivo
documents = []
for file_path in html_files:
    print(f"file path: {file_path}")
    loader = BSHTMLLoader(file_path, open_encoding="utf-8")
    documents.extend(loader.load())

for file_path in pdf_files:
    print(f"file path: {file_path}")
    loader = PyPDFLoader(file_path)
    documents.extend(loader.load())
# Ahora `documents` contiene todos los documentos cargados.
print(len(documents[0].page_content))

print(documents[0].page_content[:500])


file path: C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\autoevaluacion.html
file path: C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\carga-plan-trabajo.html
file path: C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\dashboard.html
file path: C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\desarrollo.html
file path: C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\evaluacion.html
file path: C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\evaluaciones.html
file path: C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\feedback-enviado.html
file path: C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\feedback-recibido.html
file path: C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\feedback.html
file path: C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\index.html
f

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(documents)

print(len(all_splits))
print(len(all_splits[0].page_content))
print(all_splits[10].metadata)

76
927
{'source': 'C:/Users/baiscf/repos_local/Confluence-LangChain/PMX_Manual/tenaris/es\\dashboard.html', 'title': 'Dashboard - PMX - Guía de usuarios', 'start_index': 859}


In [4]:
from langchain_chroma import Chroma
# from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

# embed = AzureOpenAIEmbeddings(
#     model=utils.config["EMB"]["MODEL"],
#     azure_endpoint=utils.config["EMB"]["ENDPOINT"],
#     api_key=utils.config["EMB"]["API_KEY"],
#     api_version=utils.config["EMB"]["API_VERSION"],
# )

os.environ["OPENAI_API_KEY"] = utils.config["EMB"]["OPENAI_API_KEY"]
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [5]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Como puedo comparar la autoevaluacion y la evaluacion?")

print(len(retrieved_docs))

print(retrieved_docs[0].page_content)

6
Es importante destacar que para accionar en esta etapa se debe tener todos los objetivos aprobados. 
 

Una vez realizada la autoevaluación se verá destacada en la escala (1) y, al colapsar, en el margen superior derecho (2).
 

 


Instancia de autoevaluación
Una vez que realices la entrevista de feedback con el supervisor y te hayan habilitado la evaluación, podrás visualizar tu autoevaluación (1) en el extremo inferior izquierdo y la evaluación del jefe (2) en la escala. 
 

 


Autoevaluación de competencias
Desde esta sección se podrá realizar la autoevaluación de las competencias. 
 


 


Competencias 	& Comportamientos
Al expandir cada una de las competencias se podrá visualizar su definición y los comportamientos asociados con una breve encuesta para completar, los cuales servirán de guía para realizar la evaluación en la escala del 1 al 5. 
 

 


Comentarios sobre competencias
Se podrán dejar comentarios en cada competencia que luego visualizará el responsable.


In [6]:
# os.environ["AZURE_OPENAI_ENDPOINT"] = utils.config["LLM"]["ENDPOINT"]
# os.environ["AZURE_OPENAI_API_KEY"] = utils.config["LLM"]["API_KEY"]
# os.environ["AZURE_OPENAI_API_VERSION"] = utils.config["LLM"]["API_VERSION"]
# os.environ["AZURE_OPENAI_DEPLOYMENT"] = utils.config["LLM"]["DEPLOYMENT"]

# from langchain_openai import AzureChatOpenAI

# llm = AzureChatOpenAI(
#     azure_endpoint=utils.config["LLM"]["ENDPOINT"],
#     azure_deployment=utils.config["LLM"]["DEPLOYMENT"],
#     openai_api_version=utils.config["LLM"]["API_VERSION"],
# )


os.environ["OPENAI_API_KEY"] = utils.config["LLM"]["OPENAI_API_KEY"]
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


We’ll use a prompt for RAG that is checked into the LangChain prompt hub (here).

In [7]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


We’ll use the LCEL Runnable protocol to define the chain, allowing us to

pipe together components and functions in a transparent way
automatically trace our chain in LangSmith
get streaming, async, and batched calling out of the box.
Here is the implementation:

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("Como puedo comparar la autoevaluacion y la evaluacion?"):
    print(chunk, end="", flush=True)

Para comparar la autoevaluación y la evaluación, puedes visualizar ambas en la escala de desempeño donde la autoevaluación se muestra en el extremo inferior izquierdo y la evaluación del jefe en la parte superior derecha. La autoevaluación permite al colaborador autoevaluarse en función de sus objetivos y competencias, mientras que la evaluación del jefe se centra en el desempeño integral del colaborador. Además, se pueden dejar comentarios en cada competencia que serán considerados en ambas evaluaciones.